<a href="https://colab.research.google.com/github/shreyasat27/pennylane-27524/blob/main/Rt_He2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 29.0 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Define the molecule
symbols = ["He", "He"]
coordinates = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.735])

# Create the molecule
mol = qml.qchem.Molecule(symbols, coordinates, basis_name="sto-3g")

# Get the number of electrons and orbitals
n_electrons = mol.n_electrons
n_orbitals = mol.n_orbitals

# Create the Hamiltonian
H, qubits = qml.qchem.molecular_hamiltonian(
    symbols,
    coordinates,
    basis="sto-3g"
)

# Define the quantum device
dev = qml.device("default.qubit", wires=qubits)

# Define the VQE ansatz
def circuit(params, wires):
    qml.BasisState(np.array([1, 1, 0, 0]), wires=wires)
    for i in range(qubits):
        qml.RY(params[i], wires=i)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])

# Define the cost function
@qml.qnode(dev)
def cost_fn(params):
    circuit(params, wires=range(qubits))
    return qml.expval(H)

# Optimize the circuit parameters
opt = qml.GradientDescentOptimizer(stepsize=0.4)
params = np.random.normal(0, np.pi, qubits)

max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):
    params, prev_energy = opt.step_and_cost(cost_fn, params)
    energy = cost_fn(params)
    conv = np.abs(energy - prev_energy)

    if n % 10 == 0:
        print(f"Iteration = {n},  Energy = {energy:.8f} Ha")

    if conv <= conv_tol:
        break

print(f"\nFinal energy: {energy:.8f} Ha")
print(f"Convergence parameter: {conv:.8f} Ha")

Iteration = 0,  Energy = -2.06990762 Ha
Iteration = 10,  Energy = -2.98401613 Ha
Iteration = 20,  Energy = -2.98817836 Ha
Iteration = 30,  Energy = -2.98871043 Ha
Iteration = 40,  Energy = -2.98877810 Ha

Final energy: -2.98878363 Ha
Convergence parameter: 0.00000099 Ha
